In [1]:
import numpy as np
from sklearn.model_selection import train_test_split

# Load data
images = np.load('images_75.npy')
labels = np.load('labels_75.npy')

# Normalize images to [0, 1] and add channel dimension
images = images.astype('float32') / 255.0
images = np.expand_dims(images, -1) 

print("Images shape:", images.shape)

# split into train, val and test sets
X_train, X_val_test, y_train, y_val_test = train_test_split(images, labels, test_size=0.2, random_state=42, shuffle=True)
X_val, X_test, y_val, y_test = train_test_split(X_val_test, y_val_test, test_size=0.5, random_state=42)


FileNotFoundError: [Errno 2] No such file or directory: 'images_75.npy'

Task 2.1 Classification

Creating 24 categories (30 minute interval)

In [3]:
def to_class_label(hours, minutes, interval=30):
    # into 30 minute intervals (interval 0 = 00:00-00:29)
    total_minutes = (hours % 12) * 60 + minutes
    return total_minutes // interval  

# Create class labels for 24 categories (30 minute intervals)
y_train_24 = to_class_label(y_train[:,0], y_train[:,1])
y_val_24 = to_class_label(y_val[:,0], y_val[:,1])
y_test_24 = to_class_label(y_test[:,0], y_test[:,1])

from tensorflow.keras.utils import to_categorical
y_train_24 = to_categorical(y_train_24, 24)
y_val_24 = to_categorical(y_val_24, 24)
y_test_24 = to_categorical(y_test_24, 24)

In [ ]:
def common_sense_err(y_true, y_pred):
    #todo adapt later to compare the original exact time values
    # Convert one-hot encoded predictions to time indices
    y_pred_classes = tf.argmax(y_pred, axis=1)
    y_true_classes = tf.argmax(y_true, axis=1)
    
    # Convert to minutes (multiply by 30 since each class represents 30 minutes)
    pred_minutes = tf.cast(y_pred_classes * 30, tf.float32)
    true_minutes = tf.cast(y_true_classes * 30, tf.float32)
    
    # Calculate absolute difference
    diff = tf.abs(pred_minutes - true_minutes)
    
    # Apply circular correction (if diff > 360, take 720 - diff)
    circular_diff = tf.minimum(diff, 720 - diff)
    
    # Return mean difference in minutes
    return tf.reduce_mean(circular_diff)

In [36]:
import tensorflow as tf
from tensorflow.keras import layers, models

# def time_classifier(X_train, y_train, X_val, y_val, pixel_size=75, num_categories=24):
#     model = models.Sequential([
#         layers.Input(shape=(pixel_size, pixel_size, 1)),
#         layers.Conv2D(32, (3, 3),padding = 'same', activation='relu'),#, input_shape=(pixel_size, pixel_size, 1)),
#         layers.BatchNormalization(),
#         layers.MaxPooling2D(pool_size=(2, 2)),
        
#         layers.Conv2D(64, (3, 3), padding = 'same', activation='relu'),
#         layers.BatchNormalization(),
#         layers.MaxPooling2D(pool_size=(2, 2)),
        
#         layers.Conv2D(128, (3, 3),padding = 'same', activation='relu'),
#         layers.BatchNormalization(),
#         layers.MaxPooling2D(pool_size=(2, 2)), 
        
#         layers.Flatten(),
#         layers.Dense(256, activation='relu'),
#         layers.Dropout(0.5),
#         layers.Dense(128, activation='relu'),
#         layers.Dropout(0.3),
#         layers.Dense(num_categories, activation='softmax')
#     ])

#     model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
#                 loss='categorical_crossentropy',
#                 metrics=['accuracy'])

#     history = model.fit(X_train, y_train, epochs=20, batch_size=64, validation_data=(X_val, y_val))

In [ ]:
# ...existing code...
# y_train_24 = y_train_24.astype('float32')
# y_val_24   = y_val_24.astype('float32')
# y_test_24  = y_test_24.astype('float32')
# print("y_train_24 dtype:", y_train_24.dtype)
# ...existing code...

y_train_24 dtype: float32


In [33]:
time_classifier(X_train, y_train_24, X_val, y_val_24, pixel_size=75, num_categories=24)

Epoch 1/50
113/113 ━━━━━━━━━━━━━━━━━━━━ 27s 220ms/step - accuracy: 0.0405 - loss: 3.1790 - val_accuracy: 0.0361 - val_loss: 3.1781
Epoch 2/50
113/113 ━━━━━━━━━━━━━━━━━━━━ 23s 206ms/step - accuracy: 0.0394 - loss: 3.1783 - val_accuracy: 0.0411 - val_loss: 3.1781
Epoch 3/50
113/113 ━━━━━━━━━━━━━━━━━━━━ 23s 205ms/step - accuracy: 0.0385 - loss: 3.1783 - val_accuracy: 0.0378 - val_loss: 3.1781
Epoch 4/50
113/113 ━━━━━━━━━━━━━━━━━━━━ 24s 213ms/step - accuracy: 0.0413 - loss: 3.1783 - val_accuracy: 0.0378 - val_loss: 3.1780
Epoch 5/50
113/113 ━━━━━━━━━━━━━━━━━━━━ 24s 208ms/step - accuracy: 0.0406 - loss: 3.1782 - val_accuracy: 0.0378 - val_loss: 3.1781
Epoch 6/50
113/113 ━━━━━━━━━━━━━━━━━━━━ 24s 210ms/step - accuracy: 0.0397 - loss: 3.1783 - val_accuracy: 0.0406 - val_loss: 3.1781
Epoch 7/50
113/113 ━━━━━━━━━━━━━━━━━━━━ 24s 216ms/step - accuracy: 0.0405 - loss: 3.1782 - val_accuracy: 0.0406 - val_loss: 3.1781
Epoch 8/50
113/113 ━━━━━━━━━━━━━━━━━━━━ 24s 213ms/step - accuracy: 0.0404 - loss: 3

(<Sequential name=sequential_12, built=True>,
 <keras.src.callbacks.history.History at 0x1eebb6415b0>)

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[28, 28]))
model.add(keras.layers.Dense(300, activation="relu"))
model.add(keras.layers.Dense(100, activation="relu"))
model.add(keras.layers.Dense(10, activation="softmax"))

model.compile(loss="sparse_categorical_crossentropy",
optimizer="sgd",
metrics=["accuracy"])

history = model.fit(X_train, y_train, epochs=30,validation_data=(X_valid, y_valid))

In [4]:
import inspect, numpy as np
import tensorflow as tf
from tensorflow.keras import models, layers, optimizers

# 2) quick build of a simple test model (no BN/Dropout) and print summary
def quick_model(X_train, y_train, X_val, y_val, pixel_size=75, num_categories=24):
    m = models.Sequential([
        layers.Input(shape=(pixel_size,pixel_size,1)),
        layers.Conv2D(32,3,padding='same',activation='relu'),
        layers.MaxPooling2D(),
        layers.Conv2D(64,3,padding='same',activation='relu'),
        layers.MaxPooling2D(),
        layers.Flatten(),
        layers.Dense(128,activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_categories,activation='softmax')
    ])
    m.compile(optimizer=optimizers.Adam(1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
    history = m.fit(X_train, y_train, epochs=150, batch_size=64, validation_data=(X_val, y_val))

#quick_model(X_train[:100], y_train_24[:100], X_val, y_val_24, pixel_size=75, num_categories=24)
quick_model(X_train, y_train_24, X_val, y_val_24, pixel_size=75, num_categories=24)

Epoch 1/150
225/225 ━━━━━━━━━━━━━━━━━━━━ 22s 91ms/step - accuracy: 0.0380 - loss: 3.1806 - val_accuracy: 0.0422 - val_loss: 3.1781
Epoch 2/150
225/225 ━━━━━━━━━━━━━━━━━━━━ 20s 90ms/step - accuracy: 0.0390 - loss: 3.1781 - val_accuracy: 0.0406 - val_loss: 3.1781
Epoch 3/150
225/225 ━━━━━━━━━━━━━━━━━━━━ 20s 90ms/step - accuracy: 0.0403 - loss: 3.1781 - val_accuracy: 0.0406 - val_loss: 3.1781
Epoch 4/150
147/225 ━━━━━━━━━━━━━━━━━━━━ 6s 86ms/step - accuracy: 0.0397 - loss: 3.1781

KeyboardInterrupt: 